In [3]:
import pulp as plp
from pulp import *
import numpy as np

In [4]:
mine_list = ['Ashley','Bedford','Consol','Dunby','Earlam','Florence','Gaston','Hopt']

mine_capacity = [300, 600, 510, 655, 575, 680, 450, 490] # (mt/yr)
mine_price = [49500, 50000, 61000, 63500, 66500, 71000, 72500, 80000] # ($/mt)

RHS = [1225, -1225, 0, 650, 720, 0]

coeff_matrix = np.array([[1,1,1,1,1,1,1,1],
                         [-1,-1,-1,-1,-1,-1,-1,-1],
                         [-1,-1,1,-1,1,-1,1,1],
                         [1,0,1,0,0,0,1,1],
                         [0,1,0,1,1,1,0,0],
                         [0.04,0.03,0.01,-0.01,-0.02,-0.03,-0.04,-0.06]])

In [5]:
n = len(mine_list)
m = len(RHS)
set_I = range(1,n+1)
set_J = range(1,m+1)

In [7]:
c = {i: mine_capacity[i-1] for i in set_I}
p = {i: mine_price[i-1] for i in set_I}

b = {j: RHS[j-1] for j in set_J}
a = {(j,i): coeff_matrix[j-1,i-1] for j in set_J for i in set_I}

In [8]:
prob = LpProblem("Supplier_selection_3")
prob.sense = LpMinimize

/opt/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [9]:
x_vars = {i: 
         LpVariable(name = "x_"+mine_list[i-1], lowBound = 0, upBound = c[i], cat=LpContinuous)
         for i in set_I}

In [10]:
constraints = {j: prob.addConstraint(
              LpConstraint(name = "Constraint_{0}".format(j), sense=-1, rhs = b[j], e = lpSum(a[j,i]*x_vars[i] for i in set_I)))
               for j in set_J}
constraints

{1: None, 2: None, 3: None, 4: None, 5: None, 6: None}

In [8]:
objective = lpSum(x_vars[i] * p[i] for i in set_I)
prob.setObjective(objective)

In [9]:
prob.solve()

1

In [10]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", plp.LpStatus[prob.status])

# Results
obj = plp.value(prob.objective)
print("The minimum coal procurement cost for next year is: ${}".format(round(obj,2)))

print("Optimal Soultion:")
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Constraints")
for c in list(prob.constraints.values()):
    print(c.name, "=", c.value())

Solution Status: Optimal
The minimum coal procurement cost for next year is: $73267500.0
Optimal Soultion:
x_Ashley = 55.0
x_Bedford = 600.0
x_Consol = 0.0
x_Dunby = 20.0
x_Earlam = 100.0
x_Florence = 0.0
x_Gaston = 450.0
x_Hopt = 0.0
Constraints
Constraint_1 = 0.0
Constraint_2 = 0.0
Constraint_3 = -125.0
Constraint_4 = -145.0
Constraint_5 = 0.0
Constraint_6 = 0.0
